In [ ]:
% pylab inline
import pandas as pd
#import numpy as np
import matplotlib as plt
import nltk
from nltk.corpus import stopwords 
import string
import heapq
import operator
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import matplotlib.pyplot as plt
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers import Dense, Dropout, Activation


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
def clean(words_string):    
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    stop_free=" ".join([i for i in words_string.lower().split() if i not in stop])
    punc_free=''.join(ch if ch not in exclude else ' ' for ch in stop_free)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

def uniteLists(list_of_lists_of_words):
    united=[]
    for list_of_words in list_of_lists_of_words:
        united+=list_of_words
    return united
def devideToTermsAndFreq(tfList):
    ans_terms=[]
    ans_freq=[]
    for key,val in tfList:
        ans_terms.append(key)
        ans_freq.append(val)
    return ans_terms,ans_freq
def createPlot(x_data,y_data,y_name,x_name,title,i,j,k,color,bartype):
    axis_font = {'family': 'serif','color':  'black','weight': 'normal','size': 18,}
    title_font = {'family': 'serif','color':  'black','weight': 'normal','size': 22,}
    font_size=16
    df=pd.DataFrame(data={x_name:x_data,y_name:y_data})
    temp = df.pivot_table(values=y_name,index=x_data)
    temp.sort_values(by=(y_name), ascending=True,inplace=True)
    ax = fig.add_subplot(i,j,k) #121
    for tick in ax.xaxis.get_major_ticks():
        tick.label1.set_fontsize(font_size)
    for tick in ax.yaxis.get_major_ticks():
        tick.label1.set_fontsize(font_size)
    ax.set_xlabel(x_name,fontdict=axis_font)
    ax.set_ylabel(y_name,fontdict=axis_font)
    ax.set_title(title,fontdict=title_font)
    temp.plot(kind=bartype,ax=ax,cmap=color)
    
def findColDiff(df,col_name1,col_name2):
    return df[[col_name1,col_name2]][data_df[col_name1] != data_df[col_name2]]

def printNewCols(start,end):
    for i in range(start,end) :
        print(data_df.iloc[i]['text'])
        print(data_df.iloc[i]['clean_text'])
        print(data_df.iloc[i]['clean_words_list'])
        print(data_df.iloc[i]['hashtags'])
        print(data_df.iloc[i]['user_tag'])
        print()

In [ ]:
import pandas as pd
data_df=pd.read_csv('gender-classifier-DFE-791531.csv',header=0,encoding = 'latin1') #iso-8859-1

In [ ]:
print(data_df.axes)

### regular expressions for text filtering

In [ ]:
emoticons_str = r"""(?:[:=;][oO\-]?[D\)\]\(\]/\\OpP])"""
html_str=r'<[^>]+>'
user_tags_str=r'(?:@[\w_]+)'
hashtags_str='#[a-zA-Z0-9]+'
url_str= r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
number_str= r'(?:(?:\d+,?)+(?:\.?\d+)?)'
another_str=r"(?:[a-zA-Z0-9#@][a-zA-Z0-9#@'\-_]+[a-zA-Z0-9#@])" # words with - and '
letters_and_tags_str='[a-z\'A-Z0-9#@][a-z\'A-Z0-9#@.-_]*'

### cleaning the data
- **'text'**- sentence per user cleaned from http links and garbage letters.
- **'hashtag'**- list of #hashtags per user.
- **'user_tag'** - list of @user_tags per user.
- **'clean_text'**- same as 'text' just without #tags and @tags.
- **'clean_words_list'**- same as 'clean_text' just splitted to list of words per user.

In [ ]:
data_df = data_df[data_df['gender'].apply(lambda x : (x=='male' or x=='female') )]
data_df['emo']= data_df['text'].apply(lambda x : re.findall(emoticons_str,x))

data_df['text']=data_df['text'].apply(lambda x :  re.sub(url_str,"",x))
data_df['text']=data_df['text'].apply(lambda x :  " ".join(re.findall(r'('+'|'.join([letters_and_tags_str,emoticons_str])+')',x)))
data_df['text']=data_df['text'].apply(lambda x :  re.sub('\'','',x))
data_df['text']=data_df['text'].apply(lambda x :  re.sub('-','',x))
data_df['text']=data_df['text'].apply(lambda x :  re.sub('_','',x))
data_df['text']=data_df['text'].apply(lambda x :  re.sub('[\s]+',' ',x))

data_df['hashtags']=data_df['text'].apply(lambda x :  re.findall(hashtags_str,x))
data_df['user_tag']=data_df['text'].apply(lambda x :  re.findall(user_tags_str,x))
data_df['clean_text']=data_df['text'].apply(lambda x :  re.sub('[#@]\S+\s*',"",x))
data_df['clean_text']=data_df['clean_text'].apply(clean)
data_df['clean_words_list']=data_df['clean_text'].apply(lambda x:re.findall('[a-zA-Z0-9]+',x))

### male/female_tf contains all the terms with there num of appearnces. 

In [ ]:
male_words=uniteLists(data_df['clean_words_list'][data_df['gender'].apply(lambda x: x=='male')])
female_words=uniteLists(data_df['clean_words_list'][data_df['gender'].apply(lambda x: x=='female')])
male_tf = Counter(male_words).most_common()
female_tf=Counter(female_words).most_common()

### plot of the num most common terms for male and female

In [ ]:
num=20
fig = plt.figure(figsize=(20,16))
terms,freq=devideToTermsAndFreq(male_tf)
createPlot(terms[:num],freq[:num],'Terms','Number of instances','male tf %s most common'%(num) ,2,1,1,'Accent','barh')
terms,freq=devideToTermsAndFreq(female_tf)
createPlot(terms[:num],freq[:num],'Terms','Number of instances','female tf %s most common'%(num),2,1,2,'Vega10_r','barh')

### male/female_hashtag_tf contains all the hashtags with there num of appearnces. 


In [ ]:
male_hashtag_words=uniteLists(data_df['hashtags'][data_df['gender'].apply(lambda x: x=='male')])
female_hashtag_words=uniteLists(data_df['hashtags'][data_df['gender'].apply(lambda x: x=='female')])
male_hashtag_tf = Counter(male_hashtag_words).most_common()
female_hashtag_tf=Counter(female_hashtag_words).most_common()

### plot for num most common #hashtags

In [ ]:
num=20
fig = plt.figure(figsize=(20,16))
terms,freq=devideToTermsAndFreq(male_hashtag_tf)
createPlot(terms[:num],freq[:num],'Terms','Number of instances','male tf %s most common #hashtags'%(num),2,1,1,'tab20c','barh')
terms,freq=devideToTermsAndFreq(female_hashtag_tf)
createPlot(terms[:num],freq[:num],'Terms','Number of instances','female tf %s most common #hashtags'%(num),2,1,2,'tab20b_r','barh')

### male/female_emo_tf contains all the emojies with there num of appearnces. 

In [ ]:
male_emo_words=uniteLists(data_df['emo'][data_df['gender'].apply(lambda x: x=='male')])
female_emo_words=uniteLists(data_df['emo'][data_df['gender'].apply(lambda x: x=='female')])
male_emo_tf = Counter(male_emo_words).most_common()
female_emo_tf=Counter(female_emo_words).most_common()

In [ ]:
num=15
fig = plt.figure(figsize=(18,16))
terms,freq=devideToTermsAndFreq(male_emo_tf)
createPlot(terms[:num],freq[:num],'emojies','Number of instances','male tf %s most common emojies'%(num),2,1,1,'Set2','barh')
terms,freq=devideToTermsAndFreq(female_emo_tf)
createPlot(terms[:num],freq[:num],'emojies','Number of instances','female tf %s most common emojies'%(num),2,1,2,'Pastel1','barh')

# /////////////////////////////////////////////////////////////////////////////////////////////////////////

# Q2 
## Train a machine learning model to predict the gender of the tweet author

In [ ]:
def changeGenderToNum(word):
    if word=='male':
        return 1 
    return 0
def changeNumToGender(num):
    if num==0:
        return 'female' 
    return 'male'

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

def benchmark(clf,x_train,y_train):
    print('_' * 80)
    print("Training: ")
    print(clf)
    scores = cross_val_score(clf, x_train, y_train, cv=5)
    t0 = time()
    clf.fit(x_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(Y_test, pred)
    print("CV accuracy:   %0.3f" % scores.mean())
    print("Y_test accuracy:   %0.3f" % score)

    print()
    clf_descr = str(clf).split('(')[0]
    return clf_descr, score, train_time, test_time


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(ngram_range=(1,2),lowercase=False)

data_df['num_gender']=data_df['gender'].apply(changeGenderToNum)

test_size=10000

word_for_voc=[]

# for i in data_df['clean_text'].tolist():
#     word_for_voc.append(i)

# for i in data_df['hashtags'].tolist():
#     for j in i:
#         word_for_voc.append(j)
        
# for i in data_df['user_tag'].tolist():
#     for j in i:
#         word_for_voc.append(j)
        
for i in data_df['terms']:
    for j in i:
        word_for_voc.append(j)
        
        
corpus_train=data_df['text'].tolist()[:test_size]

corpus_test=data_df['text'].tolist()[test_size:test_size*2]

vectorizer.fit(word_for_voc)

X_train = vectorizer.transform(corpus_train)
Y_train = data_df['num_gender'][:test_size].tolist()

X_test = vectorizer.transform(corpus_test)
Y_test = data_df['num_gender'][test_size:test_size*2].tolist()

In [ ]:
len(vectorizer.vocabulary_)
len(word_for_voc)
vectorizer.vocabulary_.keys()

In [ ]:
results = []

SVM=SGDClassifier()
NB=MultinomialNB()
KN=KNeighborsClassifier(15)
for clf, name in (
        (SVM,"SVM"),
        (KN, "kneighbors"),
        (NB,"Naive Bayes")
    ):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf,X_train,Y_train))


In [ ]:
X_train=X_train.toarray()
Y_train=np.asarray(Y_train)
X_test=X_test.toarray()
Y_test=np.asarray(Y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
numpy.random.seed(7)
# split into input (X) and output (Y) variables
# create model

model = Sequential()
model.add(Dense(X_train.shape[0], input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(X_train.shape[0], activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])
# Fit the model
model.fit(X_train, Y_train, epochs=3, batch_size=50)
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
changeNumToGender(SVM.predict(vectorizer.transform(['there is a football game this monday']).toarray())[0])

## Get most tweeted country

In [ ]:
locations = data_df['tweet_location'][data_df['tweet_location'].apply(lambda x: type(x) == str)]
locations=locations.apply(clean)
locations_tf = Counter(locations).most_common()
locations_tf

## Question 3

In [ ]:
import tweepy
from tweepy import OAuthHandler

consumer_key = 'aZgjP0NZeM4iiBaQXJcyOKLJB'
consumer_secret = 'VRLyop5JLP8Kh91CMnE0rpOKUhKEtrrVTCOozHVOd8QEFx69XX'
access_token = '326266960-9RZItG8Q4FUER0BOX4eO8mrvsvrrDWzaWiJqoens'
access_secret = 'v6UHn9puTUv5OgcDwGqf3UxfCZv6XSJZ1gB8UkXOFrnQo'

auth = OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_secret)

api = tweepy.API(auth)

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
import json
import os

In [ ]:
JSON_FILE_NAME='usa_tweets.json'
MAX_NUMBER_OF_TWEETS = 10
tweets_gathered = 0
f = open(JSON_FILE_NAME, 'w')
f.write('[')
#writer = csv.writer(f)
class MyListener(StreamListener):
    
    def on_data(self, data):
        global tweets_gathered
        if MAX_NUMBER_OF_TWEETS > tweets_gathered:
            tweets_gathered += 1
            try:
                #print(api.get_user(json.loads(data)['user']['id'])['gender']) - sadly will not work..
                f.write(data.rstrip() + ',')
                return True
            except BaseException as e:
                print("Error on_data: %s" % str(e))
            return True
        else:
            f.seek
            return False
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
# those coordinates are boxed that represents most of america - the country that we've found to have the most tweetings. 
twitter_stream.filter(locations=[-124.89,32.7,-93.07,48.99,
                                 -110.64,31.98,-84.43,47.01,
                                -99.04,26.29,-72.83,41.58,
                                -79.0,32.1,-70.6,43.69,
                                -75.3,41.06,-66.91,45.06])
f.close()
#just a fix to the json file
with open(JSON_FILE_NAME, 'rb+') as f:
    f.seek(-1, os.SEEK_END)
    f.write(b']')
    

In [ ]:
new_twitts_df = pd.read_json(path_or_buf=JSON_FILE_NAME)
display(new_twitts_df)
# Can't get twitts gender - arghhhh